In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
df = pd.read_csv('C:\\Users\\Rohan\\Desktop\\College\\Sem-5\\DA\\project\\Suicide_Detection.csv')
df.head()


,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


# Data Preprocessing

### 1. Data cleaning

In [3]:
from sklearn.model_selection import train_test_split
train_data,test_data=train_test_split(df,test_size=0.2,random_state=10)

In [4]:
import neattext.functions as nfx
# functions fro text cleaning (removing sp characters and stop words)
from tqdm import tqdm
# Progress bar library

# Takes a list of paragraphs, makes all characters lowercase, removes sp characters, removes stop words
# Returns cleaned text as array of cleaned paragraphs, list containing number of words in each cleaned paragraph
def clean_text(text):
    text_length=[]
    cleaned_text=[]
    for sent in tqdm(text):
        sent=sent.lower()
        sent=nfx.remove_special_characters(sent)
        sent=nfx.remove_stopwords(sent)
        text_length.append(len(sent.split()))
        cleaned_text.append(sent)
    return cleaned_text,text_length

In [5]:
cleaned_train_text,train_text_length=clean_text(train_data.text)
cleaned_test_text,test_text_length=clean_text(test_data.text)

100%|██████████| 46415/46415 [00:03<00:00, 12620.89it/s]


In [6]:
train_y = train_data['class']
test_y = test_data['class']

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_y = le.fit_transform(train_y)
test_y = le.fit_transform(test_y)

### 2. Stemming

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import *

stemmer = PorterStemmer()

for i in range(len(cleaned_train_text)):
    words=word_tokenize(cleaned_train_text[i])
    #produces list of words in one text piece
    words=[stemmer.stem(word) for word in words]
    
    cleaned_train_text[i] = ' '.join(words)

In [9]:
stemmer2 = PorterStemmer()

for i in range(len(cleaned_test_text)):
    words=word_tokenize(cleaned_test_text[i])
    #produces list of words in one text piece
    words=[stemmer2.stem(word) for word in words]
    
    test_text_length[i] = len(words)
    cleaned_test_text[i] = ' '.join(words)

# Feature extraction

### 1. N-Grams and vectorization

In [10]:
from gensim.models import Phrases, word2vec

sentence_stream = [doc.split() for doc in cleaned_train_text]
sentence_stream_test = [doc.split() for doc in cleaned_test_text]

In [11]:
bigram = Phrases(sentence_stream, min_count=3, threshold=2)
bigram_test = Phrases(sentence_stream_test, min_count=3, threshold=2)

In [32]:
# w2v_model = word2vec.Word2Vec(bigram[sentence_stream], vector_size = 50, min_count=1)

In [33]:
# w2v_path = "w2v_model.model"
# w2v_model.save(w2v_path)

In [13]:
w2v_model = word2vec.Word2Vec.load("w2v_model.model")

In [14]:
words = set(w2v_model.wv.index_to_key)

X_train_vect = []
for ls in cleaned_train_text:
    temp = []
    for i in ls:
        if i in words:
            temp.append(w2v_model.wv[i])
    X_train_vect.append(np.array(temp))

X_test_vect = []
for ls in cleaned_test_text:
    temp = []
    for i in ls:
        if i in words:
            temp.append(w2v_model.wv[i])
    X_test_vect.append(np.array(temp))

In [15]:
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(50, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(50, dtype=float))

### 1. SVM

In [17]:
# from sklearn.svm import SVC
# model = SVC()

# model.fit(X_train_vect_avg, train_y)

SVC()

In [2]:
import pickle
SVC_file = "model_svc.sav"
SVC_model = pickle.load(open(SVC_file, 'rb'))

In [3]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

y_pred = SVC_model.predict(X_test_vect_avg)

score = f1_score(test_y, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

score = accuracy_score(test_y, y_pred)
print('Accuracy score : {}'.format(np.round(score,4)))

score = precision_score(test_y, y_pred)
print('Precision score : {}'.format(np.round(score,4)))

score = recall_score(test_y, y_pred)
print('Recall score : {}'.format(np.round(score,4)))

NameError: name 'X_test_vect_avg' is not defined

In [29]:
# import pickle
# filename = 'model_svc.sav'
# pickle.dump(model, open(filename, 'wb'))

### 2. Random Forrest

In [52]:
# from sklearn.ensemble import RandomForestClassifier
# RF_model = RandomForestClassifier()

# RF_model.fit(X_train_vect_avg, train_y)

RandomForestClassifier()

In [17]:
import pickle
RF_file = "model_rf.sav"
RF_model = pickle.load(open(RF_file, 'rb'))

In [19]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

y_pred = RF_model.predict(X_test_vect_avg)

score = f1_score(test_y, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

score = accuracy_score(test_y, y_pred)
print('Accuracy score : {}'.format(np.round(score,4)))

score = precision_score(test_y, y_pred)
print('Precision score : {}'.format(np.round(score,4)))

score = recall_score(test_y, y_pred)
print('Recall score : {}'.format(np.round(score,4)))

F-1 score : 0.7682
Accuracy score : 0.7682
Precision score : 0.785
Recall score : 0.7387


In [54]:
# filename = 'model_rf.sav'
# pickle.dump(RF_model, open(filename, 'wb'))

### 3. Ada Boost

In [20]:
from sklearn.ensemble import AdaBoostClassifier

AB_model = AdaBoostClassifier()
AB_model.fit(X_train_vect_avg, train_y)

AdaBoostClassifier()

In [ ]:
# RF_file = "model_rf.sav"
# RF_model = pickle.load(open(RF_file, 'rb'))

In [ ]:
y_pred = AB_model.predict(X_test_vect_avg)

score = f1_score(test_y, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

score = accuracy_score(test_y, y_pred)
print('Accuracy score : {}'.format(np.round(score,4)))

score = precision_score(test_y, y_pred)
print('Precision score : {}'.format(np.round(score,4)))

score = recall_score(test_y, y_pred)
print('Recall score : {}'.format(np.round(score,4)))

In [ ]:
# filename = 'model_ab.sav'
# pickle.dump(AB_model, open(filename, 'wb'))

### 4. MLP

In [ ]:
# from sklearn.neural_network import MLPClassifier
# MLP_model = MLPClassifier()

# MLP_model.fit(X_train_vect_avg, train_y)

In [ ]:
MLP_file = "model_MLP.sav"
MLP_model = pickle.load(open(RF_file, 'rb'))

In [ ]:
y_pred = MLP_model.predict(X_test_vect_avg)

score = f1_score(test_y, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

score = accuracy_score(test_y, y_pred)
print('Accuracy score : {}'.format(np.round(score,4)))

score = precision_score(test_y, y_pred)
print('Precision score : {}'.format(np.round(score,4)))

score = recall_score(test_y, y_pred)
print('Recall score : {}'.format(np.round(score,4)))

In [ ]:
# filename = 'model_MLP.sav'
# pickle.dump(MLP_model, open(filename, 'wb'))